# Регрессия: прогноз IC50

Построение моделей для предсказания IC50 по химическим дескрипторам.

In [5]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

# Подготовка данных
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
X = df[features]
y = df["IC50, mM"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Линейная регрессия
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Случайный лес
rf = RandomForestRegressor(random_state=42)
rf_grid = GridSearchCV(rf, {'n_estimators': [100, 300], 'max_depth': [None, 10]}, cv=5, scoring='r2')
rf_grid.fit(X_train, y_train)
y_pred_rf = rf_grid.predict(X_test)

# XGBoost
xgb = XGBRegressor(random_state=42)
xgb_grid = GridSearchCV(xgb, {'n_estimators': [100, 300], 'max_depth': [3, 6]}, cv=5, scoring='r2')
xgb_grid.fit(X_train, y_train)
y_pred_xgb = xgb_grid.predict(X_test)

# LightGBM
lgb = LGBMRegressor(random_state=42)
lgb_grid = GridSearchCV(lgb, {'n_estimators': [100, 300], 'max_depth': [-1, 10]}, cv=5, scoring='r2')
lgb_grid.fit(X_train, y_train)
y_pred_lgb = lgb_grid.predict(X_test)

# Вывод результатов
models = {
    "Linear Regression": y_pred_lr,
    "Random Forest": y_pred_rf,
    "XGBoost": y_pred_xgb,
    "LightGBM": y_pred_lgb
}

for name, preds in models.items():
    print(f"{name}: R2={r2_score(y_test, preds):.3f}, MAE={mean_absolute_error(y_test, preds):.2f}, MSE={mean_squared_error(y_test, preds):.2f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002079 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 15084
[LightGBM] [Info] Number of data points in the train set: 638, number of used features: 166
[LightGBM] [Info] Start training from score 208.942374
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

# Классификация: предсказание среднего уровня избирательности (SI > медианы)

В этом разделе выполняются следующие шаги:

1. **Формирование целевой переменной**  
   - Рассчитываем медиану SI (`df["SI"].median()`).  
   - Создаём бинарный вектор `y`, где  
     - `1` = «SI > медианы» (более селективные соединения),  
     - `0` = «SI ≤ медианы» (менее селективные).

2. **Разделение выборки и масштабирование**  
   - Стратифицированный сплит на тренировочную и тестовую выборки  
     (`train_test_split(..., stratify=y, test_size=0.2)`), чтобы сохранить соотношение классов.  
   - Масштабируем признаки через `StandardScaler` для моделей, чувствительных к шкале.

3. **Обучение и оценка моделей**  
   - **LogisticRegression**, **RandomForestClassifier**, **XGBClassifier**, **LGBMClassifier**.  
   - Метрики на тестовой выборке:

   | Модель               | Accuracy | Precision | Recall | F1    | ROC AUC |
   |----------------------|----------|-----------|--------|-------|---------|
   | Logistic Regression  | 0.620    | 0.622     | 0.610  | 0.616 | 0.665   |
   | Random Forest        | 0.630    | 0.635     | 0.610  | 0.622 | 0.675   |
   | XGBoost              | 0.610    | 0.615     | 0.590  | 0.602 | 0.649   |
   | LightGBM             | 0.635    | 0.642     | 0.610  | 0.626 | 0.665   |

4. **Анализ результатов**  
   - Все модели демонстрируют скромный рост над базовым 50% (случайным угадыванием).  
   - **LightGBM** лидирует по Accuracy (0.635) и F1-score (0.626), а также по Precision (0.642).  
   - **Random Forest** показывает наилучшее ROC AUC (0.675), то есть лучшую способность ранжировать образцы по вероятности высокого SI.  
   - **XGBoost** чуть отстаёт по всем метрикам, но остаётся конкурентоспособным.

> **Вывод:** задача классификации «SI > медианы» оказалась достаточно сложной — модели достигают Accuracy ~0.62–0.64 и ROC AUC ~0.65–0.68. Среди них **LightGBM** обеспечивает лучший баланс точности и полноты, а **Random Forest** — лучшее ранжирование по вероятности. Для повышения качества стоит рассмотреть расширенный подбор гиперпараметров и дополнительные признаки.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()